In [1]:
# 기본
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
import platform
from tqdm import tqdm
import sklearn
from sklearn import linear_model
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from sklearn.model_selection import train_test_split

# crawling
import requests
import lxml.html
import sqlite3
from pandas.io import sql
from bs4 import BeautifulSoup

# 한글 폰트 설정
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)

In [37]:
건증센터 = pd.read_csv('건증센터자료_9508.csv', encoding = 'euc-kr', low_memory = False)
건증센터

,환자번호,성별,수진시나이,수진일자,신장,체중,비만도,체지방,SBP,DBP,...,uric_WBC,uric_RBC,신장_,체중_,현재혈압1,현재혈압2,당뇨여부,고혈압여부,흡연여부,음주여부
0,01295207,NaN,NaN,29JUN1995,155.9,53.6,105.1,23.8,114.0,65.0,...,5.2,4.40,155.9,53.6,0,0,4.0,2.0,1.0,2
1,07644146,F,53,12MAY1995,148.1,56.2,121.9,29.8,139.0,74.0,...,4.2,3.80,148,65.5,100,70,3.0,2.0,1.0,1
2,07654415,F,61,13APR1998,158,67.1,128.1,43.0,114.0,78.0,...,6.5,4.61,160,67,0,0,4.0,2.0,1.0,2
3,07654422,M,32,28APR1998,178.4,69.2,98.9,19.6,126.0,73.0,...,8.7,4.74,177,70,0,0,4.0,2.0,3.0,1
4,07722868,F,44,30APR2001,153.8,51.4,103.5,29.9,110.0,61.0,...,4.9,3.90,155,53,NaN,NaN,3.0,2.0,1.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71637,73502218,F,16,19AUG2004,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71638,73652377,M,49,13MAR2002,166.7,75.8,129.9,23.8,145.0,85.0,...,7.6,5.20,169,77,120,84,3.0,2.0,2.0,2
71639,74018499,F,18,07DEC2007,169.9,80.2,132.3,35.3,117.0,71.0,...,5.1,4.66,170,80,NaN,NaN,4.0,2.0,1.0,2
71640,88888888,NaN,NaN,12NOV2008,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
건증센터.columns

Index(['환자번호', '성별', '수진시나이', '수진일자', '신장', '체중', '비만도', '체지방', 'SBP', 'DBP',
       'HR', 'HB', 'platelet', 'HCT', 'calcium', 'phosphrous', 'glucose',
       'uric_acid', 'BUN', 'creatinine', 'cholesterol', 'triglyceride',
       'LDL_Chol', 'HDL_Chol', 'T_protein', 'albumin', 'T_bilirubin', 'AST',
       'ALT', 'sodium', 'potassium', 'chloride', 'SG', 'pH', 'albumin_',
       'glucose_', 'uric_WBC', 'uric_RBC', '신장_', '체중_', '현재혈압1', '현재혈압2',
       '당뇨여부', '고혈압여부', '흡연여부', '음주여부'],
      dtype='object')

------------------------------------------------

In [39]:
## 필요한 변수만 추출

col = ['체지방', '체중', '신장', 'cholesterol', '수진시나이', '흡연여부', '성별', 'SBP']
df = 건증센터[col].copy()
df

,체지방,체중,신장,cholesterol,수진시나이,흡연여부,성별,SBP
0,23.8,53.6,155.9,222.0,NaN,1.0,NaN,114.0
1,29.8,56.2,148.1,212.0,53,1.0,F,139.0
2,43.0,67.1,158,244.0,61,1.0,F,114.0
3,19.6,69.2,178.4,185.0,32,3.0,M,126.0
4,29.9,51.4,153.8,176.0,44,1.0,F,110.0
...,...,...,...,...,...,...,...,...
71637,NaN,NaN,NaN,NaN,16,NaN,F,NaN
71638,23.8,75.8,166.7,230.0,49,2.0,M,145.0
71639,35.3,80.2,169.9,169.0,18,1.0,F,117.0
71640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
# null check 

df.isna().sum()

체지방             611
체중              589
신장              589
cholesterol     828
수진시나이          1029
흡연여부           6827
성별             1030
SBP             589
dtype: int64

In [41]:
# drop all nulls

df.dropna(inplace = True)
df.reset_index(inplace = True, drop=True)
df

,체지방,체중,신장,cholesterol,수진시나이,흡연여부,성별,SBP
0,29.8,56.2,148.1,212.0,53,1.0,F,139.0
1,43.0,67.1,158,244.0,61,1.0,F,114.0
2,19.6,69.2,178.4,185.0,32,3.0,M,126.0
3,29.9,51.4,153.8,176.0,44,1.0,F,110.0
4,31.5,72.3,161.9,214.0,53,1.0,F,156.0
...,...,...,...,...,...,...,...,...
64739,16.3,52.6,163,181.0,22,1.0,M,135.0
64740,20.9,60.9,155.9,172.0,18,1.0,M,117.0
64741,37.9,83.7,168,215.0,20,2.0,F,122.0
64742,23.8,75.8,166.7,230.0,49,2.0,M,145.0


In [42]:
# recheck 

df.isna().sum()

체지방            0
체중             0
신장             0
cholesterol    0
수진시나이          0
흡연여부           0
성별             0
SBP            0
dtype: int64

In [43]:
# 단위 맞추기 - 체지방 * 1000 (kg -> g)

df['체지방'] = df['체지방']/100 * df['체중'] * 1000
df

,체지방,체중,신장,cholesterol,수진시나이,흡연여부,성별,SBP
0,16747.6,56.2,148.1,212.0,53,1.0,F,139.0
1,28853.0,67.1,158,244.0,61,1.0,F,114.0
2,13563.2,69.2,178.4,185.0,32,3.0,M,126.0
3,15368.6,51.4,153.8,176.0,44,1.0,F,110.0
4,22774.5,72.3,161.9,214.0,53,1.0,F,156.0
...,...,...,...,...,...,...,...,...
64739,8573.8,52.6,163,181.0,22,1.0,M,135.0
64740,12728.1,60.9,155.9,172.0,18,1.0,M,117.0
64741,31722.3,83.7,168,215.0,20,2.0,F,122.0
64742,18040.4,75.8,166.7,230.0,49,2.0,M,145.0


In [44]:
# 흡연여부 1~3인 사람만 추출

df = df.loc[(df['흡연여부'] >= 1) & (df['흡연여부'] <= 3)].reset_index(drop = True)
df

,체지방,체중,신장,cholesterol,수진시나이,흡연여부,성별,SBP
0,16747.6,56.2,148.1,212.0,53,1.0,F,139.0
1,28853.0,67.1,158,244.0,61,1.0,F,114.0
2,13563.2,69.2,178.4,185.0,32,3.0,M,126.0
3,15368.6,51.4,153.8,176.0,44,1.0,F,110.0
4,22774.5,72.3,161.9,214.0,53,1.0,F,156.0
...,...,...,...,...,...,...,...,...
61355,8573.8,52.6,163,181.0,22,1.0,M,135.0
61356,12728.1,60.9,155.9,172.0,18,1.0,M,117.0
61357,31722.3,83.7,168,215.0,20,2.0,F,122.0
61358,18040.4,75.8,166.7,230.0,49,2.0,M,145.0


In [45]:
# 흡연여부 check

df['흡연여부'].unique()

array([1., 3., 2.])

In [46]:
# df['흡연여부'].value_counts()

In [47]:
df[['수진시나이']]

,수진시나이
0,53
1,61
2,32
3,44
4,53
...,...
61355,22
61356,18
61357,20
61358,49


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61360 entries, 0 to 61359
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   체지방          61360 non-null  float64
 1   체중           61360 non-null  float64
 2   신장           61360 non-null  object 
 3   cholesterol  61360 non-null  float64
 4   수진시나이        61360 non-null  object 
 5   흡연여부         61360 non-null  float64
 6   성별           61360 non-null  object 
 7   SBP          61360 non-null  float64
dtypes: float64(5), object(3)
memory usage: 3.7+ MB


In [49]:
# '수진시나이' object type -> float type 변경
 
df['수진시나이'] = df['수진시나이'].astype(float, errors = 'raise')
df

,체지방,체중,신장,cholesterol,수진시나이,흡연여부,성별,SBP
0,16747.6,56.2,148.1,212.0,53.0,1.0,F,139.0
1,28853.0,67.1,158,244.0,61.0,1.0,F,114.0
2,13563.2,69.2,178.4,185.0,32.0,3.0,M,126.0
3,15368.6,51.4,153.8,176.0,44.0,1.0,F,110.0
4,22774.5,72.3,161.9,214.0,53.0,1.0,F,156.0
...,...,...,...,...,...,...,...,...
61355,8573.8,52.6,163,181.0,22.0,1.0,M,135.0
61356,12728.1,60.9,155.9,172.0,18.0,1.0,M,117.0
61357,31722.3,83.7,168,215.0,20.0,2.0,F,122.0
61358,18040.4,75.8,166.7,230.0,49.0,2.0,M,145.0


In [50]:
# '신장' object type -> float type 변경
 
df['신장'] = df['신장'].astype(float, errors = 'raise')
df

,체지방,체중,신장,cholesterol,수진시나이,흡연여부,성별,SBP
0,16747.6,56.2,148.1,212.0,53.0,1.0,F,139.0
1,28853.0,67.1,158.0,244.0,61.0,1.0,F,114.0
2,13563.2,69.2,178.4,185.0,32.0,3.0,M,126.0
3,15368.6,51.4,153.8,176.0,44.0,1.0,F,110.0
4,22774.5,72.3,161.9,214.0,53.0,1.0,F,156.0
...,...,...,...,...,...,...,...,...
61355,8573.8,52.6,163.0,181.0,22.0,1.0,M,135.0
61356,12728.1,60.9,155.9,172.0,18.0,1.0,M,117.0
61357,31722.3,83.7,168.0,215.0,20.0,2.0,F,122.0
61358,18040.4,75.8,166.7,230.0,49.0,2.0,M,145.0


In [51]:
df['성별'].unique()

array(['F', 'M'], dtype=object)

In [52]:
# dummy 변수 생성

dummy_cols = ['흡연여부', '성별']
df = pd.get_dummies(df, columns = dummy_cols)
df

,체지방,체중,신장,cholesterol,수진시나이,SBP,흡연여부_1.0,흡연여부_2.0,흡연여부_3.0,성별_F,성별_M
0,16747.6,56.2,148.1,212.0,53.0,139.0,1,0,0,1,0
1,28853.0,67.1,158.0,244.0,61.0,114.0,1,0,0,1,0
2,13563.2,69.2,178.4,185.0,32.0,126.0,0,0,1,0,1
3,15368.6,51.4,153.8,176.0,44.0,110.0,1,0,0,1,0
4,22774.5,72.3,161.9,214.0,53.0,156.0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
61355,8573.8,52.6,163.0,181.0,22.0,135.0,1,0,0,0,1
61356,12728.1,60.9,155.9,172.0,18.0,117.0,1,0,0,0,1
61357,31722.3,83.7,168.0,215.0,20.0,122.0,0,1,0,1,0
61358,18040.4,75.8,166.7,230.0,49.0,145.0,0,1,0,0,1


In [53]:
# 이상치 확인

df.describe()

,체지방,체중,신장,cholesterol,수진시나이,SBP,흡연여부_1.0,흡연여부_2.0,흡연여부_3.0,성별_F,성별_M
count,61360.000000,61360.000000,61360.000000,61360.000000,61360.000000,61360.000000,61360.000000,61360.000000,61360.000000,61360.000000,61360.000000
mean,16363.484518,63.556530,162.881227,200.983214,50.588755,129.435447,0.573924,0.169866,0.256209,0.494247,0.505753
std,6915.197623,12.608269,15.333780,37.710121,12.124154,21.002236,0.494509,0.375518,0.436542,0.499971,0.499971
min,0.000000,0.000000,0.000000,0.000000,13.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,13090.000000,55.800000,156.200000,177.000000,43.000000,115.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,16045.900000,62.700000,162.600000,199.000000,51.000000,128.000000,1.000000,0.000000,0.000000,0.000000,1.000000
75%,19212.500000,70.400000,169.100000,224.000000,58.000000,142.000000,1.000000,0.000000,1.000000,1.000000,1.000000
max,998900.100000,999.900000,1572.000000,645.000000,884.000000,999.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [54]:
# 이상치 제거를 위한 IQR 함수 작성

def IQR(df, colname): 
    quantile_25 = np.quantile(df['{}'.format(colname)], 0.25)
    quantile_75 = np.quantile(df['{}'.format(colname)],0.75)
    IQR = quantile_75 - quantile_25

    minimum = quantile_25 - 1.5 * IQR
    maximum = quantile_75 + 1.5 * IQR

    outlier_removed_df = df[(minimum <= df['{}'.format(colname)]) & (df['{}'.format(colname)] <= maximum)].reset_index(drop = True)
    
    return outlier_removed_df

In [55]:
# 체지방 이상치 - IQR로 제거

df = IQR(df, '체지방')
df

,체지방,체중,신장,cholesterol,수진시나이,SBP,흡연여부_1.0,흡연여부_2.0,흡연여부_3.0,성별_F,성별_M
0,16747.6,56.2,148.1,212.0,53.0,139.0,1,0,0,1,0
1,13563.2,69.2,178.4,185.0,32.0,126.0,0,0,1,0,1
2,15368.6,51.4,153.8,176.0,44.0,110.0,1,0,0,1,0
3,22774.5,72.3,161.9,214.0,53.0,156.0,1,0,0,1,0
4,23218.0,61.1,150.0,227.0,56.0,134.0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
60194,26343.5,89.3,177.2,208.0,20.0,146.0,1,0,0,0,1
60195,8573.8,52.6,163.0,181.0,22.0,135.0,1,0,0,0,1
60196,12728.1,60.9,155.9,172.0,18.0,117.0,1,0,0,0,1
60197,18040.4,75.8,166.7,230.0,49.0,145.0,0,1,0,0,1


In [56]:
# 체중 이상치 - IQR로 제거

df = IQR(df, '체중')
df

,체지방,체중,신장,cholesterol,수진시나이,SBP,흡연여부_1.0,흡연여부_2.0,흡연여부_3.0,성별_F,성별_M
0,16747.6,56.2,148.1,212.0,53.0,139.0,1,0,0,1,0
1,13563.2,69.2,178.4,185.0,32.0,126.0,0,0,1,0,1
2,15368.6,51.4,153.8,176.0,44.0,110.0,1,0,0,1,0
3,22774.5,72.3,161.9,214.0,53.0,156.0,1,0,0,1,0
4,23218.0,61.1,150.0,227.0,56.0,134.0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
59899,26343.5,89.3,177.2,208.0,20.0,146.0,1,0,0,0,1
59900,8573.8,52.6,163.0,181.0,22.0,135.0,1,0,0,0,1
59901,12728.1,60.9,155.9,172.0,18.0,117.0,1,0,0,0,1
59902,18040.4,75.8,166.7,230.0,49.0,145.0,0,1,0,0,1


In [57]:
# 신장 이상치 - IQR로 제거

df = IQR(df, '신장')
df

,체지방,체중,신장,cholesterol,수진시나이,SBP,흡연여부_1.0,흡연여부_2.0,흡연여부_3.0,성별_F,성별_M
0,16747.6,56.2,148.1,212.0,53.0,139.0,1,0,0,1,0
1,13563.2,69.2,178.4,185.0,32.0,126.0,0,0,1,0,1
2,15368.6,51.4,153.8,176.0,44.0,110.0,1,0,0,1,0
3,22774.5,72.3,161.9,214.0,53.0,156.0,1,0,0,1,0
4,23218.0,61.1,150.0,227.0,56.0,134.0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
59865,26343.5,89.3,177.2,208.0,20.0,146.0,1,0,0,0,1
59866,8573.8,52.6,163.0,181.0,22.0,135.0,1,0,0,0,1
59867,12728.1,60.9,155.9,172.0,18.0,117.0,1,0,0,0,1
59868,18040.4,75.8,166.7,230.0,49.0,145.0,0,1,0,0,1


In [58]:
# 콜레스테롤 이상치 - IQR로 제거

df = IQR(df, 'cholesterol')
df

,체지방,체중,신장,cholesterol,수진시나이,SBP,흡연여부_1.0,흡연여부_2.0,흡연여부_3.0,성별_F,성별_M
0,16747.6,56.2,148.1,212.0,53.0,139.0,1,0,0,1,0
1,13563.2,69.2,178.4,185.0,32.0,126.0,0,0,1,0,1
2,15368.6,51.4,153.8,176.0,44.0,110.0,1,0,0,1,0
3,22774.5,72.3,161.9,214.0,53.0,156.0,1,0,0,1,0
4,23218.0,61.1,150.0,227.0,56.0,134.0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
58907,26343.5,89.3,177.2,208.0,20.0,146.0,1,0,0,0,1
58908,8573.8,52.6,163.0,181.0,22.0,135.0,1,0,0,0,1
58909,12728.1,60.9,155.9,172.0,18.0,117.0,1,0,0,0,1
58910,18040.4,75.8,166.7,230.0,49.0,145.0,0,1,0,0,1


In [59]:
# 수진시나이 이상치 - IQR로 제거

df = IQR(df, '수진시나이')
df

,체지방,체중,신장,cholesterol,수진시나이,SBP,흡연여부_1.0,흡연여부_2.0,흡연여부_3.0,성별_F,성별_M
0,16747.6,56.2,148.1,212.0,53.0,139.0,1,0,0,1,0
1,13563.2,69.2,178.4,185.0,32.0,126.0,0,0,1,0,1
2,15368.6,51.4,153.8,176.0,44.0,110.0,1,0,0,1,0
3,22774.5,72.3,161.9,214.0,53.0,156.0,1,0,0,1,0
4,23218.0,61.1,150.0,227.0,56.0,134.0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
58760,13588.4,64.4,173.6,256.0,23.0,127.0,0,0,1,0,1
58761,12480.6,67.1,180.4,171.0,23.0,118.0,1,0,0,0,1
58762,8000.4,45.2,164.9,165.0,21.0,116.0,1,0,0,1,0
58763,8573.8,52.6,163.0,181.0,22.0,135.0,1,0,0,0,1


In [60]:
# SBP 이상치 - IQR로 제거

df = IQR(df, 'SBP')
df

,체지방,체중,신장,cholesterol,수진시나이,SBP,흡연여부_1.0,흡연여부_2.0,흡연여부_3.0,성별_F,성별_M
0,16747.6,56.2,148.1,212.0,53.0,139.0,1,0,0,1,0
1,13563.2,69.2,178.4,185.0,32.0,126.0,0,0,1,0,1
2,15368.6,51.4,153.8,176.0,44.0,110.0,1,0,0,1,0
3,22774.5,72.3,161.9,214.0,53.0,156.0,1,0,0,1,0
4,23218.0,61.1,150.0,227.0,56.0,134.0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
57823,13588.4,64.4,173.6,256.0,23.0,127.0,0,0,1,0,1
57824,12480.6,67.1,180.4,171.0,23.0,118.0,1,0,0,0,1
57825,8000.4,45.2,164.9,165.0,21.0,116.0,1,0,0,1,0
57826,8573.8,52.6,163.0,181.0,22.0,135.0,1,0,0,0,1


In [61]:
df.describe()

,체지방,체중,신장,cholesterol,수진시나이,SBP,흡연여부_1.0,흡연여부_2.0,흡연여부_3.0,성별_F,성별_M
count,57828.000000,57828.000000,57828.000000,57828.000000,57828.000000,57828.000000,57828.000000,57828.000000,57828.000000,57828.000000,57828.000000
mean,16053.222384,63.056500,162.715665,199.840458,50.456716,128.240748,0.571799,0.171457,0.256744,0.491751,0.508249
std,4385.657368,9.871839,8.262944,33.368594,10.483034,18.567473,0.494822,0.376911,0.436841,0.499936,0.499936
min,3912.300000,34.600000,137.000000,108.000000,21.000000,76.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,13024.950000,55.600000,156.300000,176.000000,43.000000,115.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15919.600000,62.400000,162.600000,199.000000,51.000000,127.000000,1.000000,0.000000,0.000000,0.000000,1.000000
75%,18954.650000,70.000000,169.000000,222.000000,58.000000,140.000000,1.000000,0.000000,1.000000,1.000000,1.000000
max,28393.200000,91.600000,188.100000,292.000000,80.000000,180.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [62]:
df.to_csv('건증센터_preprocessed.csv', encoding = 'euc-kr', index = False)